In [1]:
include("../Algebra/Gradients.jl")
include("../Algebra/Hamiltonians.jl")
include("../Algebra/Matrices.jl")
include("../Algebra/Propagators.jl")
include("../Amplitudes/Chebyshev.jl")
include("../Costs/Costs.jl")
include("../Gates/Xgate.jl")

U2 (generic function with 1 method)

In [30]:
using LinearAlgebra, Optim, Plots
using BenchmarkTools, Distributions, Random, Measures

In [31]:
ω_c = 5000
ω_q = 3000
χ = 3e-1
N = 6
amp_c = 1
amp_q = 1
T = 0.5

0.5

In [33]:
I_qubit = Matrix(I,2,2)
I_cavity = Matrix(I,N,N)
X_gate = kron(x_gate(N, Array[[3,4]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[2,3]]), I_qubit)
#X_gate = kron(x_gate(N, Array[[3,4], [5,6], [7,8]]), I_qubit)

θ = π/2.5
RZ = kron(Rz(N, Array[[3,4]], θ/2), I_qubit)
RX = kron(Rx(N, Array[[3,4]], θ/2), I_qubit)
Udue = kron(U2(N, Array[[3,4]], [θ/2 θ/3 θ/4] ), I_qubit)

sx = kron(I_cavity, [0 1; 1 0])
sy = kron(I_cavity, [0 -1im; 1im 0])

# generating matrices
a,adag,sp,sm,sz = generate_matrices(N)

interaction_transformation(t) = cis(- (ω_c * a' * a + ω_q / 2 * sz) * t)
int_transformation = interaction_transformation(T)

#generating gates
disp(α) = exp((α * adag - α' * a))
ECD(β) = disp(β / 2) * sp + disp(- β / 2) * sm
ROT(ϕ, θ) = exp(-1im * θ / 2 * ( sx * cos(ϕ) + sy * sin(ϕ) )) 
snap(x) = kron(Diagonal(exp.(-1im.*x)),I_qubit)

H_drift = χ * adag * a * sz / 2

12×12 Matrix{Float64}:
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.15   0.0   0.0   0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0   -0.15  0.0   0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.3   0.0  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0  -0.3  0.0    0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.45   0.0   0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0   -0.45  0.0   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.6   0.0  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.0  -0.6  0.0    0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.0   0.0  0.75   0.0
 0.0  0.0  0.0    0.0   0.0   0.0  0.0    0.0   0.0   0.0  0.0   -0.75

In [34]:
function cost_ecd(coefficients, unitary, N)

    L = length(coefficients)
    β_i = coefficients[1: Int(L/4)]
    β_r = coefficients[Int(L/4) + 1 : Int(L/2)]

    θs = coefficients[Int(L/2) + 1 : Int(3 * L/4)]
    ϕs = coefficients[Int(3 * L/4) + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:length(β_i)
        propagator = propagator * ROT(ϕs[i], θs[i]) * ECD(β_i[i] + β_r[i] * 1im)
    end

    return 1 - norm(tr(propagator*unitary') / N)^2
    
end


function cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, δt, coefficients, 
                                                    unitary, ω_c, ω_q, amp_q, amp_c, perturbed, sigma)

    # initialising the propagator
    dim = size(H_drift,1)
    propagator = Matrix{ComplexF64}(I,dim,dim)
    amplitude_c(t) = chebyshev_amplitude(coefficients[1:Int(length(coefficients)/2)], T, t)
    amplitude_q(t) = chebyshev_amplitude(coefficients[Int(length(coefficients)/2) + 1:end], T, t)

    if perturbed
        for l in 0:δt:T
            d_q = Normal(0, norm(amplitude_q(l)) * sigma)
            d_c = Normal(0, norm(amplitude_c(l)) * sigma)
            amp_q_perturbed = amplitude_q(l) + rand(d_q) + im*rand(d_q)
            amp_c_perturbed = amplitude_c(l) + rand(d_c) + im*rand(d_c)
            H = H_drift + amp_q * (amp_q_perturbed * sp + amp_q_perturbed' * sm) + amp_c * (amp_c_perturbed * a +  amp_c_perturbed' * adag)
            infinitesimal_propagator  = cis(- H * δt)
            propagator = infinitesimal_propagator * propagator
        end
    else 
        # time ordered product of the single exponential matrices
        for l in 0:δt:T
            H = H_drift + amp_q * (amplitude_q(l) * sp + amplitude_q(l)' * sm) + amp_c * (amplitude_c(l) * a +  amplitude_c(l)' * adag)
            infinitesimal_propagator  = cis(- H * δt)
            propagator = infinitesimal_propagator * propagator
        end
    end

    c = tr(unitary' * int_transformation * propagator)/dim
   
    return 1 - norm(c)^2
end

function snap_disp_cost_gate(coefficients, unitary, N, blocks,disp,snap)

    L = length(coefficients)
    αs = coefficients[1: blocks]
    q_d = Int(N/2)

    θs = coefficients[blocks + 1 : end]

    propagator = Matrix{ComplexF64}(I,N,N)

    for i in 1:blocks
        propagator = propagator * disp(αs[i]) * snap(θs[(i - 1) * q_d + 1 : i * q_d]) * disp(αs[i])'
    end

    return 1 - norm(tr(propagator*unitary) / N)^2
    
end

snap_disp_cost_gate (generic function with 1 method)

# Analysis SNAP & DISP

In [35]:
coeffs = [-0.0071428455433756605, -0.18659150113918432, -0.1884159450841208, -0.4888884512046601, -0.18620386166850653, 2.3051028286120157, -2.774266791132211, -4.648288377291242, 1.7792536374685253, 3.1103313265197943, 4.288877483034881, 1.0615533755510103, -0.6667686882945248, -0.32673888415233693, 1.227141866396891, -1.8037070753964626, 1.320965841456551, -1.845890231580013, -3.525506270643454, 5.151330044152118, -2.2099608424483685, -2.8316625593112956, 3.190922170584367, 1.802651335371791, 1.1352101101857224, 0.30499316185891534, 2.9954665130470706, -4.142003699560306, 1.4106736034079386, 0.7884336203750318, 3.6950501911851608, 0.10164367844344295, 0.7579624945915933, -2.8599443291882696, 0.012194556477236117]
f(x) = snap_disp_cost_gate(x, X_gate, N * 2, 5,disp,snap)
println(f(coeffs))
println(length(coeffs))

non_perturbed_snap = f(coeffs)
perturbed_costs_2_snap = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-2)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_2_snap, f(opt_perturbed))
end
perturbed_costs_3_snap = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-3)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_3_snap, f(opt_perturbed))
end
perturbed_costs_4_snap = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-4)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_4_snap, f(opt_perturbed))
end
perturbed_costs_5_snap = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-5)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_5_snap, f(opt_perturbed))
end

3.9672603176499344e-5
35


# Analysis PO

In [36]:
h(x) = cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, x, X_gate, ω_c, ω_q, 10, 10, false, 1e-3)

h (generic function with 1 method)

In [40]:
coeffs = [-0.5897800212401615 + 0.46136325340423284im, -0.8960838978518413 + 0.47738735795403114im, 0.9338746433078257 + 0.2854200779021651im, -1.8717231056019303 + 1.1841088164889584im, -0.09201097116377728 - 0.5160976440271168im, -0.36595876629420954 - 0.8407352383526249im, 0.7915173512937284 - 0.5959111155523337im, 0.9753094924690314 - 1.0275441854037308im, 0.3734083112539267 - 0.7689519645115519im, 1.2272278687459985 - 0.8067964691285543im, 0.1977247693081553 - 1.2049713343728974im, -0.8639250806911528 - 1.1228725673477078im, 1.0807890582792483 - 1.8928903233236813im, 1.07682356604934 - 0.7487968064290523im, -0.5246852984347219 - 1.466451672077681im, -0.2778816133535102 - 0.22893551361539177im, 0.36562521330069186 - 0.47923286794777986im, 1.1620067153726834 + 0.8636537937592125im, -0.01719124882468189 - 0.7537667832490684im, -2.1741754992367106 - 0.03214339938421833im, 1.5000036966037675 - 0.9015605268411766im, -2.529383627165414 - 0.7479754776145724im, -1.1297828220709116 - 0.770362232983346im, 1.0303897428425697 - 0.294907849310252im, 0.000777949112539651 - 0.00028031032599552027im, -1.2641970823623097 - 0.116384003249834im, 0.16413787969034815 - 1.0894852178556962im, 0.22707759168340838 - 1.6832131362663048im, 0.2234513896952926 - 0.14122585035479956im, 0.3758641289977396 + 0.3118171820620781im, -0.6347586303207485 - 0.4535126409194866im, 0.897530464498016 - 1.6029861544055068im, 0.43274931906051217 + 0.6602403140121361im, -0.2605991509874922 + 0.37444191650600545im, -0.4877086897520587 - 1.03962352365116im, -0.662093749019389 + 0.7178902627185183im, 0.0659647974240543 - 1.1342597198442752im, 0.8806411974952728 + 0.5166460007172635im, -0.49173126551258667 - 0.3557582241559857im, 0.07495088312311433 - 1.3017824841240233im, 0.8230872673280053 - 0.721876575488717im, -0.27177849856034353 + 0.18256705092312964im, -0.9010108751578303 + 0.6561284786972803im, 0.7978953799001206 - 0.41056025449804084im, 0.006187248066795746 + 0.5765651166496989im, 0.43165082974199426 + 1.2740393339416im, 0.39261801806392393 - 1.3436102737285056im, 0.8515791569553032 + 0.41478259772086545im, 0.607824941679106 + 0.6992638744496897im, 0.46406199566484 + 0.7968199683198464im, 0.917174076953976 + 0.43227422495140433im, 0.3832263153168602 - 0.0582378479537239im, -0.3232519536556208 - 0.26778334982098373im, -1.0067657415592997 + 0.7555805868060371im, 0.9198642178987387 - 0.9494345191337088im, 0.010547440816143252 + 0.8335801936446917im, -0.8989438639633263 - 0.9331857301528247im, -0.7721243417920413 - 0.45917736431863143im, 0.23727118031457914 - 1.1705232026895467im, 0.6531422196840351 - 0.9528788964799366im]
println(cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, false, 1e-3))
println(length(coeffs))

non_perturbed_po = cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, false, 1e-3)
perturbed_costs_2_po = []
for i in 1:50
    push!(perturbed_costs_2_po,cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, true, 1e-2))
end
perturbed_costs_3_po = []
for i in 1:50
    push!(perturbed_costs_3_po,cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, true, 1e-3))
end
perturbed_costs_4_po = []
for i in 1:50
    push!(perturbed_costs_4_po,cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, true, 1e-4))
end
perturbed_costs_5_po = []
for i in 1:50
    push!(perturbed_costs_5_po,cost_from_0_dispersive_pulse_optimisation(H_drift, sp, sm, a, adag, T, T/50, coeffs, X_gate, ω_c, ω_q, 10, 10, true, 1e-5))
end


3.099232268655161e-5
60


# Analysis ECD

In [41]:
f(x) = cost_ecd(x, X_gate, N * 2)
coeffs = [-0.0004542676443930162, 0.463983647397694, 0.8057216770701501, -1.0428890370778945, 1.0346041989778685, -1.0157409297084963, 1.1627718509375004, 0.8273520897955802, -0.3378097081719437, -0.4732980401480722, 0.053638073286200474, 0.8483176157657761, -0.46973137212810023, 1.7894961336762418, -0.6922724467578238, 0.4695872237170776, 0.5838701624687844, -0.8945126477550567, -1.3000449144130886, 1.269921526672431, 0.5950553727888436, -1.2887219823143914, 1.9916801090271152, -0.8325979274812602, 0.33410011790416094, 0.4568158146865538, -1.8013829899629394, -0.7740019297235085, 0.5283866182141913, -1.3194525807314879, -0.2541090695209986, 1.0804729002707214, 0.23513161430656956, -0.44026001327855774, 0.9483196475557526, 1.1194651019121664, 0.8744583387932068, -0.9349632455167786, -0.15390988834511232, -1.027267538541953, 0.47074883644059967, 0.43943595266929253, -0.7831038880243666, -0.12888593432883086, 0.9067739813491015, -0.4377164676309994, -0.4570877180555607, -0.10943856682273508, -0.4160759936891394, 0.2573049583166998, -0.5993221115339713, 2.2363250019336123, 1.0684251543889898, 0.13890924678782113, -0.12901008134233122, -0.1046231489968068, -1.4116774858968821, 0.5163145342616844, -0.6755842648649988, 0.08455809371556997, -0.9861410532780095, 0.1513049158073435, -0.9771523756834243, 0.5845592322223188, 1.9618251073566135, -0.0884710560136058, -0.6222479286217307, 0.4093263994352731, -1.3363563968912668, -0.43954371539536913, -1.346124781175214, 0.28728999905749303, -2.4918288789800145, 0.8498757539404913, 1.822872566909653, -2.1266019143174963, -1.0802631471734832, 2.4357124557093814, -0.7451843693222856, 2.9435710877991035, 1.9553484108995187, -1.7132757702006374, 1.6736520047951906, -1.4641843217627375, -3.925931314751142, 2.0311348296342406, -2.363680148549257, -2.6788448654501598, -0.6992508737027372, -2.51978696581134, -0.7619356923682787, -0.4062049765745829, 1.7675687581250752, -2.3680218735820837, 2.4150436050549824, -1.5013297445836238, 2.2698603959034283, 2.0470044498304896, -1.5456549743112256, 3.0331253122364803, -1.4752600681954917, -2.6410516954445353, 0.3921096105194526, 0.7182061541822801, 0.8277832968056804, -2.024661757952073, 1.879312576056947, -0.9674816200159004, -3.457967514137439, -2.761695735869275, 0.18382728098491374, -1.5050386729365968, 1.0264421287249175, 0.6951862936910773, -1.0710233925275556, 0.7684882260092742, -1.895162341632579, 1.4358511001882956, -1.4332641630668956, -2.0435702417371995, -1.8536425315276677, 1.0928446843479878, 2.0927384217429386, -2.5478382277890472, 0.34391219519961685, 0.34828333695250174, -1.9365552811263802, -2.4423642543606876, 0.21141537518603365, -2.7174476129260414, 2.596192648051221, 1.2937767737181536, -1.496988573679427, 0.7425414458977176, 2.6919217136418414, 1.489313583498254, -1.308755509516141, -2.1996074289729335, 1.8359983338217727, 2.8356977558106498, -0.004623223165573614, 1.7023673749463637, 2.0954893792216462, -1.4723771128612517]
println(f(coeffs))
println(length(coeffs))

non_perturbed = f(coeffs)
perturbed_costs_2 = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-2)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_2, f(opt_perturbed))
end
perturbed_costs_3 = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-3)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_3, f(opt_perturbed))
end
perturbed_costs_4 = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-4)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_4, f(opt_perturbed))
end
perturbed_costs_5 = []
for i in 1:50
    opt_perturbed = ComplexF64[]

    for coeff in coeffs
        d = Normal(0, norm(coeff) * 1e-5)
        perturbed = coeff + (rand(d))
        push!(opt_perturbed,perturbed)
    end
    push!(perturbed_costs_5, f(opt_perturbed))
end

3.567522164471537e-5
144


In [47]:
plot([1e-6, 1e-5,1e-4,1e-3, 1e-2],[non_perturbed, mean(perturbed_costs_5),mean(perturbed_costs_4),mean(perturbed_costs_3),mean(perturbed_costs_2)], ribbon = [0, std(perturbed_costs_5),std(perturbed_costs_4),std(perturbed_costs_3), std(perturbed_costs_2)],label= "ECD", xlabel="β", ylabel="Infidelity", fmt = :PDF, size=(800,400), margin=5mm, formatter = :plain, xtickfontsize=14,ytickfontsize=14,xguidefontsize=16,yguidefontsize=16,legendfontsize=14, linestyle=:dash, marker = :circle, yaxis=:log, xaxis=:log, legend=:topleft, markersize = 6)
plot!([1e-6, 1e-5,1e-4,1e-3, 1e-2],[non_perturbed_snap, mean(perturbed_costs_5_snap),mean(perturbed_costs_4_snap),mean(perturbed_costs_3_snap),mean(perturbed_costs_2_snap)], ribbon = [0, std(perturbed_costs_5_snap),std(perturbed_costs_4_snap),std(perturbed_costs_3_snap), std(perturbed_costs_2_snap)],label= "SNAP and Displacement", xlabel="β", ylabel="Infidelity", fmt = :PDF, margin=5mm, formatter = :plain, xtickfontsize=14,ytickfontsize=14,xguidefontsize=16,yguidefontsize=16,legendfontsize=14, linestyle=:dash, marker = :circle, yaxis=:log, xaxis=:log, markersize = 6)
plot!([1e-6, 1e-5,1e-4,1e-3, 1e-2],[non_perturbed_po,mean(perturbed_costs_5_po),mean(perturbed_costs_4_po),mean(perturbed_costs_3_po),mean(perturbed_costs_2_po)], ribbon = [0, std(perturbed_costs_5_po),std(perturbed_costs_4_po),std(perturbed_costs_3_po), std(perturbed_costs_2_po)],label= "Pulse Optimisation", xlabel="β", ylabel="Infidelity", fmt = :PDF, margin=5mm, formatter = :plain, xtickfontsize=14,ytickfontsize=14,xguidefontsize=16,yguidefontsize=16,legendfontsize=14, linestyle=:dash, marker = :circle, yaxis=:log, xaxis=:log, markersize = 6)
savefig("stability_analysis.pdf")

"/Users/andreamaestri/Desktop/Thesis_julia/Opt3DQalgs/BTgates/src/plots_and_analysis/stability_analysis.pdf"